<h2>FT003</h2>
<p> En este apartado se trata la base de datos FT003 </p>

In [ ]:
#Cargamos los paquetes que vamos a necesitar
import pandas as pd
import numpy as np
import pyreadstat
pd.pandas.set_option('display.max_columns', None)

In [ ]:
# Se carga el FT003 para 2020 y 2021 completo desde el archivo .dta
FT003_2021_2021 = pd.read_stata('C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/Informe_Pagos/FT003 20202021.dta')

In [ ]:
# Se carga el FT003 para diciembre de 2019 desde el archivo .dta
FT003_2019 = pd.read_stata('C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/Informe_Pagos/FT003 122019.dta')

In [ ]:
# Se concatenan la bases de datos 
frames = [FT003_2021_2021, FT003_2019]
FT003 = pd.concat(frames, ignore_index=True)

In [ ]:
# Se eliminan la bases que no se necesitan para liberar espacio en la memoria
del [FT003_2021_2021, FT003_2019, frames]

In [ ]:
# Se reemplazan los nombres de las columnas para facilitar el tratamiento
FT003.rename(columns={'Nit': 'nit_eps', 'Año': 'ano', 'Periodo': 'mes', 'lineaNegocio': 'linea','tipoIdDeudor': 'tipoiddeudor', 'idDeudor': 'nitproveedor', 'conceptoDeudores': 'concepto', 'tipoDeuda': 'tipodeuda', 'medicionPosterior': 'medicion', 'cxcPendientesRadicar': 'cxcsinradicar','cxcNoVencidas': 'cxcnovencidas', 'cxcMora30Dias': 'cxc30', 'cxcMora60Dias': 'cxc60', 'cxcMora90Dias': 'cxc90','cxcMora180Dias': 'cxc180', 'cxcMora360Dias': 'cxc360', 'cxcMoraMayor360Dias': 'cxcmayor360'}, inplace=True)

In [ ]:
# Se transforman los datos para su correcta identificación
FT003[['nit_eps', 'RazonSocial', 'ano', 'mes', 'linea', 'tipoiddeudor',
       'nitproveedor', 'dvDeudor', 'nombreDeudor', 'codigoMunicipio',
       'concepto', 'tipodeuda', 'medicion']] = FT003[['nit_eps', 'RazonSocial', 'ano', 'mes', 'linea', 'tipoiddeudor',
       'nitproveedor', 'dvDeudor', 'nombreDeudor', 'codigoMunicipio',
       'concepto', 'tipodeuda', 'medicion']].astype(str)

FT003[['cxcsinradicar', 'cxcnovencidas',
       'valorMercado', 'cxc30', 'cxc60', 'cxc90', 'cxc180', 'cxc360',
       'cxcmayor360', 'deterioro30Dias', 'deterioro60Dias', 'deterioro90Dias',
       'deterioro180Dias', 'deterioro360Dias', 'deterioroMayor360Dias',
       'ajuste', 'saldo']] = FT003[['cxcsinradicar', 'cxcnovencidas',
       'valorMercado', 'cxc30', 'cxc60', 'cxc90', 'cxc180', 'cxc360',
       'cxcmayor360', 'deterioro30Dias', 'deterioro60Dias', 'deterioro90Dias',
       'deterioro180Dias', 'deterioro360Dias', 'deterioroMayor360Dias',
       'ajuste', 'saldo']].astype(float)

In [ ]:
# Se filtra la línea de negocio de interés. En este caso corresponde a la línea 1
FT003 = FT003.loc[FT003['linea'] == '1']

In [ ]:
FT003.columns

In [ ]:
# Se cuenta el número de duplicados para hacer la respectiva corrección del tipo id deudor
df_deudor = FT003.groupby(['nitproveedor', 'nombreDeudor']).size().reset_index().rename(columns={0:'Conteo_FT03'})

In [ ]:
# Para guardar la base de datos lista para ser tratada en formato Excel y CSV
df_deudor.to_excel(r'C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/df_deudor_FT003_V2.xlsx', index = False)

In [ ]:
# Se crea la columna Cuentas por Cobrar (CXC) sumando las columnas cxcsinradicar, cxcnovencidas, cxc30, cxc60, cxc90, cxc180, cxc360 y cxcmayor360
CXC = FT003['cxcsinradicar'] + FT003['cxcnovencidas'] + FT003['cxc30'] + FT003['cxc60'] + FT003['cxc90'] + FT003['cxc180'] + FT003['cxc360'] + FT003['cxcmayor360']
FT003['CXC'] = CXC

In [ ]:
# Para hacer la suma de CXC por NIT EPS, tipo de proveedor, NIT proveedor, Mes y Año
FT003 = FT003.groupby(['nit_eps', 'tipoiddeudor', 'nitproveedor', 'mes', 'ano']).agg({'CXC': 'sum'}).reset_index()

In [ ]:
# Se comprueba que no haya duplicados cuando se filtra por NIT EPS, tipo de deudor, NIT proveedor, Mes y Año. Para esto se cuenta el número de duplicados
duplicadosFT003 = FT003[FT003.duplicated(['nit_eps', 'tipoiddeudor', 'nitproveedor', 'mes', 'ano'], keep='last')]
print("El número de registros duplicados es:", duplicadosFT003.nit_eps.count(), sep='\n')

In [ ]:
# Se genera una nueva columna extrayendo el primer caracter del ID nitproveedor
FT003['d1'] = FT003['nitproveedor'].str[:1]

In [ ]:
# Se genera una nueva columna con la cuenta del numero de componentes que tiene el ID nitproveedor
FT003['largo'] = FT003.nitproveedor.str.len()

In [ ]:
# Se cuenta el número de duplicados para hacer la respectiva corrección del tipo id deudor
df_tipoiddeudor = FT003.groupby(['nit_eps', 'nitproveedor', 'mes', 'ano']).size().reset_index().rename(columns={0:'d'})
# df_tipoiddeudor.sort_values(by='d', ascending=False)

In [ ]:
# Se crea la unión de del FT003 con la cuenta de repetidos según NIT EPS, NIT proveedor, mes y año
FT003 = FT003.merge(df_tipoiddeudor, left_on=['nit_eps', 'nitproveedor', 'mes', 'ano'], right_on=['nit_eps', 'nitproveedor', 'mes', 'ano'], how='left')

In [ ]:
# Se limpia la clasificación por Tipo de deudor
FT003['tipoiddeudor_corregido'] = FT003['tipoiddeudor'] # Se crea la columna con categoria NA como base
# Condiciones
FT003.loc[
       (FT003['tipoiddeudor_corregido'] == 'NA') &
       (FT003['largo'] == 10) & # Para signar el tipo de deudor CC
       (FT003['d1'] == '1'), 
       'tipoiddeudor_corregido'] = 'CC' 

FT003.loc[
       (FT003['tipoiddeudor_corregido'] == 'NA') &
       (FT003['largo'] == 9) & # Para signar el tipo de deudor NI
       ((FT003['d1'] == '7') |
       (FT003['d1'] == '8') |
       (FT003['d1'] == '9')),  
       'tipoiddeudor_corregido'] = 'NI' 

FT003.loc[
       (FT003['tipoiddeudor_corregido'] == 'NA') &
       (FT003['tipoiddeudor_corregido'] != 'NI'), # Para completar el tipo de deudor CC
       'tipoiddeudor_corregido'] = 'CC' 

FT003.loc[
       (FT003['tipoiddeudor_corregido'] == 'NA') &
       (FT003['largo'] != 9), # Para completar el tipo de deudor CC
       'tipoiddeudor_corregido'] = 'CC' 

In [ ]:
# Para conocer las diferencias entre las correciones
print(FT003.tipoiddeudor.value_counts(dropna=True))
print(FT003.tipoiddeudor_corregido.value_counts())

In [ ]:
# Para hacer la suma de CXC por NIT EPS, tipo de deudor corregido, NIT proveedor, Mes y Año
FT003 = FT003.groupby(['nit_eps', 'tipoiddeudor', 'nitproveedor', 'mes', 'ano']).agg({'CXC': 'sum'}).reset_index()

In [ ]:
FT003[['ano', 'mes']] = FT003[['ano', 'mes']].astype(int)

In [ ]:
# Se crea el rango de meses necesarios en cada mes
fecha_ideal=range(1, 12 + 1)
# Se rellena con ceros la información para los meses faltantes
FT003 = FT003.set_index('mes').groupby(['nit_eps', 'tipoiddeudor', 'nitproveedor', 'ano']).apply(lambda x: x.reindex(index=fecha_ideal, fill_value=0)).drop(['nit_eps', 'tipoiddeudor', 'nitproveedor', 'ano'], 1).reset_index()

In [ ]:

# Se encuentran las observaciones que cumplen las condiciones
df_filtered = FT003[
    (FT003['ano'] == 2019) & (FT003['mes'] == 1) |
    (FT003['ano'] == 2019) & (FT003['mes'] == 2) |
    (FT003['ano'] == 2019) & (FT003['mes'] == 3) |
    (FT003['ano'] == 2019) & (FT003['mes'] == 4) |
    (FT003['ano'] == 2019) & (FT003['mes'] == 5) |
    (FT003['ano'] == 2019) & (FT003['mes'] == 6) |
    (FT003['ano'] == 2019) & (FT003['mes'] == 7) |
    (FT003['ano'] == 2019) & (FT003['mes'] == 8) |
    (FT003['ano'] == 2019) & (FT003['mes'] == 9) |
    (FT003['ano'] == 2019) & (FT003['mes'] == 10) |
    (FT003['ano'] == 2019) & (FT003['mes'] == 11)
    ].index

# Se eliminan las observaciones que cumplen las condiciones anteriore
FT003 = FT003.drop(df_filtered)

In [ ]:
# Se completan los años y los meses para las observaciones que no presentaron ninguna observación en un año de interés. Por ejemplo, a aquellos sujetos que reportaron cifras en 2019, pero no en 2020, se les completa el año 2020 para poder calcular la diferencia entre diciembre de 2019 y enero de 2020
# Se convierte la variable en mes y año
FT003[['ano', 'mes']] = FT003[['ano', 'mes']].astype(float)
FT003['periodo'] = pd.to_datetime(FT003.ano*10000 + FT003.mes*100 + 1, format='%Y%m%d')
fecha_ideal = pd.date_range('12-01-2019','05-01-2021',freq='M')

In [ ]:
FT003_dif = FT003.groupby(['nit_eps', 'tipoiddeudor', 'nitproveedor', pd.Grouper(key='periodo', freq='M')])[['CXC']].sum().reset_index()

In [ ]:
FT003_dif = FT003_dif.set_index('periodo').groupby(['nit_eps', 'tipoiddeudor', 'nitproveedor']).apply(lambda x: x.reindex(index=fecha_ideal, fill_value=0)).drop(['nit_eps', 'tipoiddeudor', 'nitproveedor'], 1).reset_index()

In [ ]:
# Se reemplazan los nombres de las columnas para facilitar el tratamiento
FT003_dif.rename(columns={'level_3': 'periodo'}, inplace=True)

In [ ]:
FT003_dif = FT003_dif.set_index(['nit_eps', 'nitproveedor', 'tipoiddeudor', 'periodo'])

In [ ]:
# Se calcula la diferencia mensual para conocer el valor mensual de cada cuenta ya que por defecto vienen agregadas
FT003_dif = FT003_dif.diff().fillna(0).reset_index()

In [ ]:
# Se reemplazan los nombres de las columnas para facilitar el tratamiento
FT003_dif.rename(columns={'CXC': 'DCXCmes'}, inplace=True)

#  Se extrae el mes y el año para porterior left join con el agregado
FT003_dif['ano'] = FT003_dif['periodo'].dt.year
FT003_dif['mes'] = FT003_dif['periodo'].dt.month

#Se transforman las variables a STR para mejorar su manipulación
FT003[['ano', 'mes']] = FT003[['ano', 'mes']].astype(int)

# Se eliminan las columnas que no son de interés
FT003 = FT003.drop(['periodo'], axis=1)
FT003_dif = FT003_dif.drop(['periodo'], axis=1)

In [ ]:
# Se unen las dos bases de datos para obtener la información de webgironoupc (Giro Directo - No_UPC - Compra de Cartera) y FT005 en una sola base de datos. Desagregando el valor de VCausado y el VPagado para cada mes
FT003 = pd.merge(FT003_dif, FT003, how='left', left_on=['nit_eps', 'tipoiddeudor', 'nitproveedor', 'mes', 'ano'], right_on=['nit_eps', 'tipoiddeudor', 'nitproveedor', 'mes', 'ano'])

In [ ]:
# Se encuentran las observaciones que cumplen las condiciones
df_filtered = FT003[
    (FT003['ano'] == 2019)
    ].index

# Se eliminan las observaciones que cumplen las condiciones anteriore
FT003 = FT003.drop(df_filtered)

In [ ]:
# Se carga la base de datos prestadores.dta para su posterior pega
prestadores = pd.read_stata('C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/prestadores.dta')

In [ ]:
# Se comprueba que no haya duplicados cuando se filtra por Nitproveedor. Para esto se cuenta el número de duplicados
duplicados_prestadores = prestadores[prestadores.duplicated(['nitproveedor'], keep='last')]
print("El número de registros duplicados es:", duplicados_prestadores.nitproveedor.count(), sep='\n')

In [ ]:
# Se unen las dos bases de datos para obtener la información de FT005 con webgironoupc (Giro Directo - No_UPC - Compra de Cartera) y prestadores
FT003_pres= pd.merge(FT003, prestadores, how='left', left_on=['nitproveedor'], right_on=['nitproveedor'])

In [ ]:
# Se carga la base de datos operadores.dta para su posterior pega
operadores = pd.read_stata('C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/operadores.dta')

In [ ]:
# Se comprueba que no haya duplicados cuando se filtra por Nitproveedor. Para esto se cuenta el número de duplicados
duplicados_operadores = operadores[operadores.duplicated(['nitproveedor'], keep='last')]
print("El número de registros duplicados es:", duplicados_operadores.nitproveedor.count(), sep='\n')

In [ ]:
# Se unen las dos bases de datos para obtener la información de FT005 con webgironoupc (Giro Directo - No_UPC - Compra de Cartera) y prestadores
FT003_pres_ope= pd.merge(FT003_pres, operadores, how='left', left_on=['nitproveedor'], right_on=['nitproveedor'])

In [ ]:
# Se reemplazan los nombres de las columnas para facilitar el tratamiento
FT003_pres_ope.rename(columns={'tipoiddeudor_corregido': 'tipoiddeudor'}, inplace=True)

In [ ]:
# Se eliminan la bases que no se necesitan para liberar espacio en la memoria
del [FT003, FT003_pres, operadores, duplicados_operadores, prestadores, duplicados_prestadores, duplicadosFT003, CXC]

In [ ]:
# Se crea la columna noID tomando como referencia la columna clpr_nombre
FT003_pres_ope['noID'] = FT003_pres_ope['clpr_nombre'] 

In [ ]:
# Se establecen las condiciones para establecer correctamente el noID

FT003_pres_ope.loc[(FT003_pres_ope['nit_eps'] == FT003_pres_ope['nitproveedor']), # Se reemplaza por EPS todo aquel proveedor que sea una EPS
       'noID'] = 'EPS'

FT003_pres_ope.loc[(FT003_pres_ope['nitproveedor'] == '"901037916"') | # Se reemplazan los NIT de la nación
       (FT003_pres_ope['nitproveedor'] == "900047282") |
       (FT003_pres_ope['nitproveedor'] == "899999014") |
       (FT003_pres_ope['nitproveedor'] == "900474727") |
       (FT003_pres_ope['nitproveedor'] == "900462447"),  
       'noID'] = 'Nacion' 

FT003_pres_ope.loc[FT003_pres_ope['noID'].isnull(), 'noID'] = 'Otro' # Para rellenar los vacios con "Otro"

FT003_pres_ope.loc[(FT003_pres_ope['clpr_nombre'] == 'IPS') & # Para corregir el IPS cuando Tipo es vacío, por IPS-OL-DM-GIM
       (FT003_pres_ope['tipo'].notnull()), 
       'noID'] = 'IPS-OL-DM-GIM'

FT003_pres_ope.loc[(FT003_pres_ope['tipo'].notnull()) & # Para dejar el Tipo cuando se tiene OSD u Otro
       ((FT003_pres_ope['noID'] == 'OSD') |
       (FT003_pres_ope['noID'] == 'Otro')), 
       'noID'] = FT003_pres_ope['tipo']

FT003_pres_ope.loc[(FT003_pres_ope['noID'] == 'Otro') & # Para cambiar el noID por Persona Natural si en Tipo Proveedor se tiene CC y CE
       ((FT003_pres_ope['tipoiddeudor'] == 'CC') |
       (FT003_pres_ope['tipoiddeudor'] == 'CE')), 
       'noID'] = 'Persona Natural'

FT003_pres_ope.loc[(FT003_pres_ope['departamento'].isnull()) & # Para asignar "Otro" al departamento si está vacío y si es una IPS o IPS Pública
       ((FT003_pres_ope['noID'] == 'IPS Pública') |
       (FT003_pres_ope['noID'] == 'IPS')), 
       'departamento'] = 'Otro'

In [ ]:
# Se convierte la variable en mes y año
FT003_pres_ope[['ano', 'mes']] = FT003_pres_ope[['ano', 'mes']].astype(float).astype(int)

In [ ]:
# Se agrega la información por NIT_EPS, Nitproveedor, noID, Departamento, Ano, Mes para las variables VPagado, VCausado, PW, Giro, No_UPC, Compra_Cartera
FT003_pres_ope = FT003_pres_ope.groupby(['nit_eps', 'nitproveedor', 'noID','mes', 'ano']).agg({'CXC':'sum', 'DCXCmes':'sum'}).reset_index()

In [ ]:
# Para guardar la base de datos lista para ser tratada en formato Excel y CSV

# Para guardar la base de datos lista para ser tratada en formato Excel y CSV
# FT003_pres_ope.to_excel(r'C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/cxcprestador_2020_2021.xlsx', index = False)
FT003_pres_ope.to_csv(r'C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/cxcprestador_2020_2021.csv', index = False, encoding='utf-8-sig')

In [ ]:
# Se eliminan la bases que no se necesitan para liberar espacio en la memoria
del [FT003_pres_ope, FT003_dif, df_filtered, fecha_ideal]